In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.cross_validation import train_test_split
from sklearn import neighbors
from collections import defaultdict
import xgboost as xgb
import datetime
import gc

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def add_date_features(df):
    df["date"] = pd.to_datetime(df["time_stamp"])
    df["date_year"] = df["date"].dt.year
    df["date_month"] = df["date"].dt.month
    df["date_day"] = df["date"].dt.day
    df["date_hour"] = df["date"].dt.hour
    df["date_min"] = df["date"].dt.minute
    return df

In [3]:
shop_info = pd.read_csv('train_ccf_first_round_shop_info.csv')
user_shop_info = pd.read_csv('train_ccf_first_round_user_shop_behavior.csv')
testA_B = pd.read_csv('AB_test_evaluation_public.csv') 
shop_info['category_id'] = shop_info['category_id'].str.split('c_', expand=True)[1].astype(np.int32)

In [4]:
testA_B

,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos
0,118742,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,2017-09-06 13:10,118.191907,32.855858,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,2017-09-06 17:40,119.192110,32.424667,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,2017-09-03 12:10,120.612201,34.055249,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,2017-09-02 20:40,116.861989,40.326858,b_19882704|-77|false;b_2241462|-49|false;b_585...
5,118747,u_30099328,m_4079,2017-09-01 13:20,117.365336,40.638300,b_33503892|-50|false;b_30058118|-66|false;b_75...
6,118748,u_30099795,m_2333,2017-09-06 12:20,120.494443,36.455076,b_44437637|-70|false;b_56327744|-67|false;b_45...
7,118749,u_30099923,m_1175,2017-09-07 21:10,120.744362,30.814718,b_8763036|-91|false;b_8324226|-48|false;b_3340...
8,118750,u_30100284,m_1409,2017-09-03 10:30,122.309345,32.086750,b_2159244|-82|false;b_23387932|-93|false;b_169...
9,118751,u_30100446,m_3517,2017-09-02 19:00,104.417599,31.286157,b_34749981|-68|false;b_45712058|-86|false;b_55...


In [5]:
user_shop_info

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...
5,u_2733,s_3686420,2017-08-28 13:10,122.595277,31.581963,b_13784698|-75|false;b_7390858|-86|false;b_358...
6,u_2848,s_89874,2017-08-11 14:20,104.815313,30.962847,b_975810|-33|true;b_975807|-41|false;b_3506304...
7,u_3063,s_939447,2017-08-06 14:50,117.913713,39.589681,b_15263246|-67|false;b_10120963|-76|false;b_41...
8,u_3063,s_939447,2017-08-02 17:30,117.913765,39.589699,b_37749132|-77|false;b_25781349|-63|false;b_25...
9,u_3063,s_3698335,2017-08-02 18:30,117.914182,39.589539,b_39846139|-73|false;b_25673942|-61|false;b_25...


In [5]:
shop_info

,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,4,122.346736,31.833507,57,m_690
1,s_133,6,121.134362,31.197511,58,m_6587
2,s_251,38,121.000505,30.907667,34,m_5892
3,s_372,30,119.864982,26.659876,44,m_625
4,s_456,26,122.594243,31.581499,44,m_3839
5,s_462,47,115.088969,30.690787,58,m_3739
6,s_469,50,121.963123,31.652801,64,m_1293
7,s_496,47,120.746132,30.815237,62,m_1175
8,s_729,6,119.326859,32.644347,58,m_2182
9,s_775,26,121.000467,30.908378,44,m_5892


In [52]:
cat_shop = defaultdict(lambda : [])
for line in shop_info.values:
    cat_shop[line[1]].append(line[0])

cat_shop

defaultdict(<function __main__.<lambda>>,
            {1: ['s_865',
              's_1017',
              's_1587',
              's_2482',
              's_7741',
              's_35458',
              's_35640',
              's_46669',
              's_47447',
              's_49656',
              's_51309',
              's_51357',
              's_57565',
              's_63309',
              's_65714',
              's_66450',
              's_68678',
              's_69183',
              's_75197',
              's_78967',
              's_81527',
              's_82879',
              's_83151',
              's_84388',
              's_94942',
              's_96187',
              's_96223',
              's_107788',
              's_110658',
              's_117882',
              's_118705',
              's_130911',
              's_133772',
              's_140773',
              's_146498',
              's_147610',
              's_165957',
              's_167638',


In [73]:
category = []
for line in user_shop_info.values:
    for count in cat_shop.items():
        if line[0] in count[1]:
            category.append(count[0])

In [83]:
catg = pd.DataFrame(category, columns=['category'])
catg

,cat
0,38
1,38
2,30
3,27
4,6
5,34
6,49
7,28
8,28
9,25


In [89]:
user_shop_info['category'] = catg
user_shop_info

,shop_id,longitude,latitude,date_day,date_hour,date_min,wifi_id,wifi_signal,wifi_flag,catg
0,s_2871718,122.308289,32.088039,6,21,20,6396479,-55,0,38
1,s_2871718,122.308159,32.087971,6,21,20,6396479,-57,0,38
2,s_181637,117.365257,40.638214,2,13,10,2485110,-52,0,30
3,s_609470,121.134453,31.197416,13,12,30,30424471,-60,0,27
4,s_3816766,122.255867,31.351320,25,19,50,39004148,-58,0,6
5,s_3686420,122.595276,31.581963,28,13,10,21608835,-44,0,34
6,s_89874,104.815315,30.962847,11,14,20,975810,-33,1,49
7,s_939447,117.913712,39.589680,6,14,50,13067253,-60,0,28
8,s_939447,117.913765,39.589699,2,17,30,42938093,-34,0,28
9,s_3698335,117.914185,39.589539,2,18,30,25674236,-51,0,25


In [26]:
shop_info.category_id.unique()

array([  4,   6,  38,  30,  26,  47,  50,  10,  24,  33,  29,   1,  48,
        56,  36,  42,   2,  28,  49,  27,  14,  11,  15,  12,  39,  34,
        43,  55,  13,  19,  46,  53,  44,  25,  59,  51,   7,   3,   9,
        17,  23,  41,  22,  40,  52,   5,  18,  20,  62,  54, 136, 134,
       132, 135,  82,  16, 129, 128, 127, 131,  45,   8, 133,  21, 345,
       130, 112], dtype=int64)

In [6]:
add_date_features(user_shop_info)
lb = LabelBinarizer()
train_wifi_id = []
train_wifi_signal = []
train_wifi_flag = []
for line in user_shop_info.values:
    wifi = sorted([wifi.split('|') for wifi in line[5].split(';')],key=lambda x:int(x[1]),reverse=True)[0]
    train_wifi_id.append(wifi[0])
    train_wifi_signal.append(wifi[1])
    train_wifi_flag.append(wifi[2])
    
user_shop_info['wifi_id'] = train_wifi_id
user_shop_info['wifi_signal'] = train_wifi_signal
user_shop_info['wifi_flag'] = train_wifi_flag
user_shop_info['wifi_flag'] = lb.fit_transform(user_shop_info['wifi_flag'])
user_shop_info['wifi_signal'] = user_shop_info['wifi_signal'].astype(np.int32)
user_shop_info['wifi_id'] = user_shop_info['wifi_id'].str.split('b_', expand=True)[1].astype(np.int32)


user_shop_info['date_day'] = user_shop_info['date_day'].astype(np.int32)
user_shop_info['date_hour'] = user_shop_info['date_hour'].astype(np.int32)
user_shop_info['date_min'] = user_shop_info['date_min'].astype(np.int32)
user_shop_info['longitude'] = user_shop_info['longitude'].astype(np.float32)
user_shop_info['latitude'] = user_shop_info['latitude'].astype(np.float32)


user_shop_info = user_shop_info.drop(['wifi_infos', 'date', 'time_stamp', 'date_year', 'date_month', 'user_id'], axis=1)
user_shop_info

,shop_id,longitude,latitude,date_day,date_hour,date_min,wifi_id,wifi_signal,wifi_flag
0,s_2871718,122.308289,32.088039,6,21,20,6396479,-55,0
1,s_2871718,122.308159,32.087971,6,21,20,6396479,-57,0
2,s_181637,117.365257,40.638214,2,13,10,2485110,-52,0
3,s_609470,121.134453,31.197416,13,12,30,30424471,-60,0
4,s_3816766,122.255867,31.351320,25,19,50,39004148,-58,0
5,s_3686420,122.595276,31.581963,28,13,10,21608835,-44,0
6,s_89874,104.815315,30.962847,11,14,20,975810,-33,1
7,s_939447,117.913712,39.589680,6,14,50,13067253,-60,0
8,s_939447,117.913765,39.589699,2,17,30,42938093,-34,0
9,s_3698335,117.914185,39.589539,2,18,30,25674236,-51,0


In [15]:
user_shop_info['wifi_signal'] = np.abs(user_shop_info['wifi_signal'])

In [7]:
add_date_features(testA_B)
lb = LabelBinarizer()
test_wifi_id = []
test_wifi_signal = []
test_wifi_flag = []
for line in testA_B.values:
    wifi = sorted([wifi.split('|') for wifi in line[6].split(';')],key=lambda x:int(x[1]),reverse=True)[0]
    test_wifi_id.append(wifi[0])
    test_wifi_signal.append(wifi[1])
    test_wifi_flag.append(wifi[2])


testA_B['wifi_id'] = test_wifi_id
testA_B['wifi_signal'] = test_wifi_signal
testA_B['wifi_flag'] = test_wifi_flag
testA_B['wifi_flag'] = lb.fit_transform(testA_B['wifi_flag'])
testA_B['wifi_id'] = testA_B['wifi_id'].str.split('b_', expand=True)[1].astype(np.int32)
testA_B['wifi_signal'] = testA_B['wifi_signal'].astype(np.int32)

testA_B['date_day'] = testA_B['date_day'].astype(np.int32)
testA_B['date_hour'] = testA_B['date_hour'].astype(np.int32)
testA_B['date_min'] = testA_B['date_min'].astype(np.int32)
testA_B['longitude'] = testA_B['longitude'].astype(np.float32)
testA_B['latitude'] = testA_B['latitude'].astype(np.float32)


testA_B = testA_B.drop(['wifi_infos', 'date', 'time_stamp', 'date_year', 'date_month', 'user_id'], axis=1)

testA_B

,row_id,mall_id,longitude,latitude,date_day,date_hour,date_min,wifi_id,wifi_signal,wifi_flag
0,118742,m_3916,122.141014,39.818848,5,13,0,37756289,-53,0
1,118743,m_5085,118.191910,32.855858,6,13,10,49853639,-68,0
2,118744,m_4033,119.192108,32.424667,6,17,40,40924464,-54,0
3,118745,m_4515,120.612198,34.055248,3,12,10,52869345,-42,0
4,118746,m_7168,116.861992,40.326859,2,20,40,29284311,-42,0
5,118747,m_4079,117.365334,40.638302,1,13,20,33919931,-49,0
6,118748,m_2333,120.494446,36.455074,6,12,20,16608380,-50,0
7,118749,m_1175,120.744362,30.814718,7,21,10,8324226,-48,0
8,118750,m_1409,122.309341,32.086750,3,10,30,11907579,-56,0
9,118751,m_3517,104.417603,31.286158,2,19,0,32048186,-44,0


In [9]:
mall_ids = list( shop_info.ix[:,'mall_id'].unique() )

In [8]:

shop_info['mall_id'] = shop_info['mall_id'].str.split('m_', expand=True)[1].astype(np.int32)
shop_info

,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,4,122.346736,31.833507,57,690
1,s_133,6,121.134362,31.197511,58,6587
2,s_251,38,121.000505,30.907667,34,5892
3,s_372,30,119.864982,26.659876,44,625
4,s_456,26,122.594243,31.581499,44,3839
5,s_462,47,115.088969,30.690787,58,3739
6,s_469,50,121.963123,31.652801,64,1293
7,s_496,47,120.746132,30.815237,62,1175
8,s_729,6,119.326859,32.644347,58,2182
9,s_775,26,121.000467,30.908378,44,5892


In [56]:
for i in range(10):   
    price_shop = defaultdict(lambda : defaultdict(lambda : []))
    price_shop[i][i].append('a')

price_shop

defaultdict(<function __main__.<lambda>>,
            {9: defaultdict(<function __main__.<lambda>.<locals>.<lambda>>,
                         {9: ['a']})})

In [11]:
price_shop = defaultdict(lambda : defaultdict(lambda : []))
for mall_id in mall_ids[:10]:
    shop_ids_by_mall_id = shop_info[shop_info.mall_id == mall_id]
    #data_by_mall_id = user_shop_info[user_shop_info['shop_id'].isin(shop_ids_by_mall_id['shop_id'])]
    per_mall_avg_price = shop_ids_by_mall_id.price.mean()
    price_shop[per_mall_avg_price][mall_id].append(shop_ids_by_mall_id.shop_id.values)
    
price_shop.items()

dict_items([(48.755319148936174, defaultdict(<function <lambda>.<locals>.<lambda> at 0x000000000971BE18>, {6587: [array(['s_133', 's_828', 's_3735', 's_3780', 's_4721', 's_7893', 's_8178',
       's_22504', 's_48916', 's_53742', 's_61508', 's_64615', 's_65583',
       's_72613', 's_73027', 's_86032', 's_134834', 's_142276', 's_143658',
       's_213073', 's_219161', 's_251652', 's_264236', 's_266452',
       's_275457', 's_296184', 's_296847', 's_298569', 's_345695',
       's_352840', 's_375770', 's_382398', 's_385005', 's_403680',
       's_412825', 's_428339', 's_436475', 's_446984', 's_447707',
       's_464841', 's_474193', 's_480414', 's_483185', 's_504234',
       's_553018', 's_595077', 's_609470', 's_646953', 's_889824',
       's_1119772', 's_1150421', 's_1203412', 's_1516725', 's_1946554',
       's_2683921', 's_2819490', 's_2944347', 's_2953013', 's_3024639',
       's_3063168', 's_3131512', 's_3302515', 's_3312468', 's_3401200',
       's_3465042', 's_3484894', 's_3545750'

In [31]:
shop_ids_by_mall_id

,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,4,122.346736,31.833507,57,m_690
29,s_1587,1,122.347116,31.834759,60,m_690
34,s_1756,27,122.346919,31.833567,44,m_690
41,s_2258,38,122.346777,31.833647,50,m_690
43,s_2265,38,122.346113,31.834622,50,m_690
55,s_3221,49,122.346859,31.833882,73,m_690
75,s_3697,30,122.346763,31.834160,56,m_690
155,s_5947,26,122.346802,31.833697,52,m_690
199,s_9147,29,122.346026,31.833803,50,m_690
259,s_12639,49,122.347065,31.834226,59,m_690


In [12]:
test_data_by_mall_id = testA_B[testA_B['mall_id'] == mall_ids[0]]
row_and_shop_list_resuslt = pd.DataFrame()
row_and_shop_list_resuslt['row_id'] = test_data_by_mall_id.row_id
    
row_and_shop_list_resuslt
    


,row_id
237,118979
290,119032
326,119068
407,119149
413,119155
440,119182
473,119215
537,119279
553,119295
743,119485


In [11]:
count = 0

result = pd.DataFrame()
row_and_shop_list_resuslt = pd.DataFrame()

for mall_id in mall_ids[:98]:
    
    row_and_shop_list_resuslt.drop(row_and_shop_list_resuslt.index,inplace=True)
    count1 = 0 
    
    count = count + 1
    if(count % 10 == 0):
        print(count)
    shop_ids_by_mall_id = shop_info[shop_info.mall_id == mall_id]
    data_by_mall_id = user_shop_info[user_shop_info['shop_id'].isin(shop_ids_by_mall_id['shop_id'])]
    
    test_data_by_mall_id = testA_B[testA_B['mall_id'] == mall_id]
    row_and_shop_list_resuslt['row_id'] = test_data_by_mall_id.row_id
    
    test_data_by_mall_id = test_data_by_mall_id.drop(['mall_id', 'row_id'], axis=1)
    X = data_by_mall_id.ix[:, 1:]
    y = data_by_mall_id.ix[:, :1]
    x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.19)  
    clf = neighbors.KNeighborsClassifier(algorithm='kd_tree', n_neighbors=5)  
    clf.fit(X, y)
    
    
    answer1 = clf.predict(test_data_by_mall_id)
    
#    for j in range(len(answer1)):
#\        if answer1[j] == np.array(y_valid)[j]:
#            count1 = count1 + 1
#
#    print(1.0 * count1 / len(answer1))       
#    print(answer1)

    row_and_shop_list_resuslt['shop_id'] = pd.Series(clf.predict(test_data_by_mall_id)).values
    
    result = result.append(row_and_shop_list_resuslt, ignore_index=True)
        

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:25: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


10
20
30


KeyboardInterrupt: 

In [27]:
len(testA_B)

483931

In [25]:
result.to_csv('result.csv', index=False)

In [16]:
count = 0

result = pd.DataFrame()
for mall_id in mall_ids[:98]:
    count1 = 0 
    
    count = count + 1
    if(count % 10 == 0):
        print(count)
    shop_ids_by_mall_id = shop_info[shop_info.mall_id == mall_id]
    data_by_mall_id = user_shop_info[user_shop_info['shop_id'].isin(shop_ids_by_mall_id['shop_id'])]
    
    #test_data_by_mall_id = testA_B[testA_B['mall_id'] == mall_id]
    #test_data_by_mall_id = test_data_by_mall_id.drop(['mall_id', 'row_id'], axis=1)
    X = data_by_mall_id.ix[:, 1:]
    y = data_by_mall_id.ix[:, :1]
    x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.19)  
   # clf = neighbors.KNeighborsClassifier(algorithm='kd_tree', n_neighbors=5)  
    clf = MultinomialNB().fit(x_train, y_train) 
   # clf.fit(x_train, y_train)
    answer1 = clf.predict(x_valid)
    
    for j in range(len(answer1)):
        if answer1[j] == np.array(y_valid)[j]:
            count1 = count1 + 1

    print(1.0 * count1 / len(answer1))       
 #   print(len(answer1))
   # mid_result = pd.DataFrame(pd.Series(clf.predict(test_data_by_mall_id)).values, columns=['shop_id'])
   # result = result.append(mid_result, ignore_index=True)
        #print(result.append(pd.Series(mid_result), ignore_index=True))  
        

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.11028257456828885
0.12315478700970055
0.17165492957746478
0.0996488147497805
0.07729805013927576
0.13123268698060941
0.14926783183750592
0.1772887323943662
0.1337333904843549
10
0.1266025641025641
0.11193072482360487
0.08307322929171669
0.08271266170129361
0.11787819253438114
0.2149122807017544
0.07830188679245283
0.14494074749316319
0.029551630434782608
0.14955640050697086
20
0.07784583971413987
0.16815476190476192
0.08303249097472924
0.1093011035207567
0.05391373801916933
0.29598051157125455
0.0763615946097698
0.14833040421792618
0.24789915966386555
0.10743061772605192
30
0.1762295081967213
0.17211538461538461
0.12778782399035563
0.14418811002661935
0.17260940032414912
0.3767268862911796
0.25331724969843183
0.16593886462882096
0.1509433962264151
0.10730088495575221
40
0.1137009664582149
0.1254422643936957
0.0971540726202159
0.13448090371167296
0.22757475083056478
0.09528535980148883
0.07663029687896473
0.13111962537249894
0.1945970695970696
0.13114754098360656
50
0.1354300385109114

ValueError: Input X must be non-negative

In [ ]:
count = 0
result = pd.DataFrame()
row_and_shop_list_resuslt = pd.DataFrame()

for mall_id in mall_ids[:98]:
    count1 = 0 
    
    count = count + 1
    if(count % 10 == 0):
        print(count)
    shop_ids_by_mall_id = shop_info[shop_info.mall_id == mall_id]
    data_by_mall_id = user_shop_info[user_shop_info['shop_id'].isin(shop_ids_by_mall_id['shop_id'])]
    
    test_data_by_mall_id = testA_B[testA_B['mall_id'] == mall_id]
    row_and_shop_list_resuslt['row_id'] = test_data_by_mall_id.row_id
    
    test_data_by_mall_id = test_data_by_mall_id.drop(['mall_id', 'row_id'], axis=1)
    
    X = data_by_mall_id.ix[:, 1:]
    y = data_by_mall_id.ix[:, :1]
    y['shop_id'] = y['shop_id'].str.split('s_', expand=True)[1].astype(np.int32)
    #x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.19)
    
    forest = RandomForestClassifier(n_estimators=150, random_state=1)
    multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
    multi_target_forest.fit(X, y)
    answer1 = multi_target_forest.predict(test_data_by_mall_id)
    
#    for j in range(len(answer1)):
#        if answer1[j] == np.array(y_valid)[j]:
#            count1 = count1 + 1
#
#    print(1.0 * count1 / len(answer1)) 
#    print(answer1)
    row_and_shop_list_resuslt['shop_id'] = pd.Series(clf.predict(test_data_by_mall_id)).values
    
    result = result.append(row_and_shop_list_resuslt, ignore_index=True)



c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
Exception in thread Thread-80:
Traceback (most recent call last):
  File "c:\users\scarlet\appdata\local\programs\python\python35\lib\threading.py", line 923, in _bootstrap_inner
    self.run()
  File "c:\users\scarlet\appdata\local\programs\python\python35\lib\threading.py", line 871, in run
    self._target(*self._args, **self._kwargs)
  File "c:\users\scarlet\appdata\local\programs\python\python35\lib\multiprocessing\pool.py", line 429, in _handle_results
    task = get()
  File "c:\users\scarlet\appdata\local\programs\python\python35\lib\multiprocessing\connection.py", line 250, in recv
    buf = self._recv_bytes()
  

In [35]:
a = pd.DataFrame(answer1, columns=['shop_id'])
a

,shop_id
0,633878
1,16473
2,405021
3,1425466
4,3578421
5,1217813
6,211875
7,149642
8,532828
9,43301


In [53]:
pd.Series(['s_']).str.cat([pd.Series(a.shop_id[0].astype(np.str))], sep='')

0    s_633878
dtype: object

In [ ]:
count = 0
result = pd.DataFrame()
for mall_id in mall_ids[:1]:
    count1 = 0 
    
    count = count + 1
    if(count % 10 == 0):
        print(count)
    shop_ids_by_mall_id = shop_info[shop_info.mall_id == mall_id]
    data_by_mall_id = user_shop_info[user_shop_info['shop_id'].isin(shop_ids_by_mall_id['shop_id'])]
    
    #test_data_by_mall_id = testA_B[testA_B['mall_id'] == mall_id]
    #test_data_by_mall_id = test_data_by_mall_id.drop(['mall_id', 'row_id'], axis=1)
    X = data_by_mall_id.ix[:, 1:]
    y = data_by_mall_id.ix[:, :1]
    y['shop_id'] = y['shop_id'].str.split('s_', expand=True)[1].astype(np.int32)
    x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.19)
    
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dtest = xgb.DMatrix(x_valid, label=y_valid)
    
    n_class = len(y_train.shop_id.unique())
    watchlist = [(dtrain, 'train'),(dtest, 'test')]
    
    params = {'max_depth':3, 'eta':0.1, 'num_class':n_class, 'objective':'multi:softmax','nthread':4, 'silent':0}
    num_boost_round = 5
    bst = xgb.train(params, dtrain, num_boost_round, watchlist)
    
    mid_result = pd.DataFrame(pd.Series(bst.predict(dtest)).values, columns=['shop_id'])
    result = result.append(mid_result, ignore_index=True)
        #print(result.append(pd.Series(mid_result), ignore_index=True))    
        
        
result

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
sub = pd.DataFrame({'row_id' : testA_B['row_id'].values, 'shop_id' : result['shop_id'].values}, columns=['row_id', 'shop_id'])
sub

,row_id,shop_id
0,118742,s_1756
1,118743,s_9147
2,118744,s_9147
3,118745,s_70432
4,118746,s_455479
5,118747,s_405021
6,118748,s_298312
7,118749,s_9147
8,118750,s_2789464
9,118751,s_149642


In [21]:
from datetime import datetime
sub.to_csv('result.csv', index=False)

In [12]:
ab = pd.read_csv('result.csv')
ab

,row_id,shop_id
0,1,s_3597
1,2,s_3597
2,3,s_1611
3,4,s_3480
4,5,s_374984
5,6,s_1872
6,7,s_469
7,8,s_3480
8,9,s_1872
9,10,s_3842


In [20]:
 n_classes = len(data_with_mall_id.ix[:,'shop_id'].unique())
_n_samples = data_with_mall_id.shape[0]
n_classes

94

In [74]:
mall = pd.DataFrame(shop_info['mall_id'].unique(), columns=['ID'])

In [145]:
len(mall)

97

In [75]:
mall_1 = shop_info[shop_info['mall_id'] == mall['ID'][0]] 

In [76]:
mall_1

,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,c_4,122.346736,31.833507,57,m_690
29,s_1587,c_1,122.347116,31.834759,60,m_690
34,s_1756,c_27,122.346919,31.833567,44,m_690
41,s_2258,c_38,122.346777,31.833647,50,m_690
43,s_2265,c_38,122.346113,31.834622,50,m_690
55,s_3221,c_49,122.346859,31.833882,73,m_690
75,s_3697,c_30,122.346763,31.834160,56,m_690
155,s_5947,c_26,122.346802,31.833697,52,m_690
199,s_9147,c_29,122.346026,31.833803,50,m_690
259,s_12639,c_49,122.347065,31.834226,59,m_690


In [77]:
add_date_features(user_shop_info)
user_shop_info

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos,date,date_year,date_month,date_day,date_hour,date_min
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...,2017-08-06 21:20:00,2017,8,6,21,20
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...,2017-08-06 21:20:00,2017,8,6,21,20
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...,2017-08-02 13:10:00,2017,8,2,13,10
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...,2017-08-13 12:30:00,2017,8,13,12,30
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...,2017-08-25 19:50:00,2017,8,25,19,50
5,u_2733,s_3686420,2017-08-28 13:10,122.595277,31.581963,b_13784698|-75|false;b_7390858|-86|false;b_358...,2017-08-28 13:10:00,2017,8,28,13,10
6,u_2848,s_89874,2017-08-11 14:20,104.815313,30.962847,b_975810|-33|true;b_975807|-41|false;b_3506304...,2017-08-11 14:20:00,2017,8,11,14,20
7,u_3063,s_939447,2017-08-06 14:50,117.913713,39.589681,b_15263246|-67|false;b_10120963|-76|false;b_41...,2017-08-06 14:50:00,2017,8,6,14,50
8,u_3063,s_939447,2017-08-02 17:30,117.913765,39.589699,b_37749132|-77|false;b_25781349|-63|false;b_25...,2017-08-02 17:30:00,2017,8,2,17,30
9,u_3063,s_3698335,2017-08-02 18:30,117.914182,39.589539,b_39846139|-73|false;b_25673942|-61|false;b_25...,2017-08-02 18:30:00,2017,8,2,18,30


In [78]:
user_shop_info = user_shop_info.drop(['date', 'time_stamp', 'date_year', 'date_month', 'user_id'], axis=1)
user_shop_info['wifi_infos'] = user_shop_info['wifi_infos'].str.split(';', expand=True)[0]
user_shop_info['wifi_id'] = user_shop_info['wifi_infos'].str.split('|', expand=True)[0]
user_shop_info['wifi_signal'] = user_shop_info['wifi_infos'].str.split('|', expand=True)[1]
user_shop_info['wifi_flag'] = user_shop_info['wifi_infos'].str.split('|', expand=True)[2]

lb = LabelBinarizer()
user_shop_info['wifi_flag'] = lb.fit_transform(user_shop_info['wifi_flag'])
user_shop_info = user_shop_info.drop(['wifi_infos'], axis=1)


user_shop_info

,shop_id,longitude,latitude,date_day,date_hour,date_min,wifi_id,wifi_signal,wifi_flag
0,s_2871718,122.308291,32.088040,6,21,20,b_6396480,-67,0
1,s_2871718,122.308162,32.087970,6,21,20,b_6396480,-67,0
2,s_181637,117.365255,40.638214,2,13,10,b_8006367,-78,0
3,s_609470,121.134451,31.197416,13,12,30,b_26250579,-73,0
4,s_3816766,122.255867,31.351320,25,19,50,b_39004150,-66,0
5,s_3686420,122.595277,31.581963,28,13,10,b_13784698,-75,0
6,s_89874,104.815313,30.962847,11,14,20,b_975810,-33,1
7,s_939447,117.913713,39.589681,6,14,50,b_15263246,-67,0
8,s_939447,117.913765,39.589699,2,17,30,b_37749132,-77,0
9,s_3698335,117.914182,39.589539,2,18,30,b_39846139,-73,0


In [121]:
user_shop_info['wifi_signal'] = user_shop_info['wifi_signal'].astype(np.int32)
user_shop_info['date_day'] = user_shop_info['date_day'].astype(np.int32)
user_shop_info['date_hour'] = user_shop_info['date_hour'].astype(np.int32)
user_shop_info['date_min'] = user_shop_info['date_min'].astype(np.int32)
user_shop_info['longitude'] = user_shop_info['longitude'].astype(np.float32)
user_shop_info['latitude'] = user_shop_info['latitude'].astype(np.float32)
user_shop_info['wifi_id'] = user_shop_info['wifi_id'].str.split('b_', expand=True)[1].astype(np.int32)
user_shop_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138015 entries, 0 to 1138014
Data columns (total 9 columns):
shop_id        1138015 non-null object
longitude      1138015 non-null float32
latitude       1138015 non-null float32
date_day       1138015 non-null int32
date_hour      1138015 non-null int32
date_min       1138015 non-null int32
wifi_id        1138015 non-null int32
wifi_signal    1138015 non-null int32
wifi_flag      1138015 non-null int32
dtypes: float32(2), int32(6), object(1)
memory usage: 43.4+ MB


In [122]:
x_train_1 = user_shop_info[user_shop_info['shop_id'].isin(mall_1['shop_id'])] 

In [123]:
x_train_1

,shop_id,longitude,latitude,date_day,date_hour,date_min,wifi_id,wifi_signal,wifi_flag
10,s_149642,122.346924,31.834587,15,9,40,33798074,-90,0
43,s_149642,122.346939,31.834576,6,13,50,25145248,-55,0
56,s_767494,122.347305,31.833839,10,12,30,4121837,-53,0
57,s_5947,122.347343,31.833675,10,12,30,15853307,-69,0
112,s_9147,122.346779,31.833582,12,8,10,18988859,-62,1
113,s_9147,122.346550,31.834223,13,9,0,22076003,-79,0
134,s_298312,122.346680,31.834848,11,18,20,7446873,-68,0
145,s_1425466,122.346092,31.834360,10,17,30,13587603,-63,0
194,s_149642,122.346924,31.834623,6,19,20,22076003,-64,0
195,s_2833968,122.347031,31.835049,6,20,30,15412066,-79,0


In [127]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_1, y_train, test_size = 0.19)  

In [138]:
clf = neighbors.KNeighborsClassifier(algorithm='kd_tree', n_neighbors=5)  
clf.fit(x_train, y_train)

KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [139]:
answer = clf.predict(test)

In [140]:
answer

array(['s_240723', 's_9147', 's_9147', ..., 's_12639', 's_3345001',
       's_1091614'], dtype=object)

In [141]:
answer1 = clf.predict(x_valid)

In [142]:
len(answer1)

5096

In [143]:
len(y_valid)

5096

In [144]:
count = 0
for i in range(len(answer1)):
    if answer1[i] == y_valid[i]:
        count = count + 1
        
print(count/len(answer1))

0.4872448979591837
